## NBA Career Prediction Experiment

In [733]:
experiment_label = 'xgb03'

### Aim:
* 


### Findings:

Retain -ve values, apply SMOTE & under sampling pipeline, search on roc_auc.
Results train, val auc:
* 


## Set up

In [734]:
import pandas as pd
import numpy as np
from joblib import dump, load  # simpler than pickle!
import matplotlib.pyplot as plt
import seaborn as sns

## Data

In [735]:
#load data

data_path = '../data/raw/uts-advdsi-nba-career-prediction'

train_raw = pd.read_csv(data_path + '/train.csv')
test_raw = pd.read_csv(data_path + '/test.csv')


In [736]:
#shapes & head

print(train_raw.shape)
print(test_raw.shape)

train_raw.head()

(8000, 22)
(3799, 21)


,Id_old,Id,GP,MIN,PTS,FGM,FGA,FG%,3P Made,3PA,...,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,TARGET_5Yrs
0,10556,3799,80,24.3,7.8,3.0,6.4,45.7,0.1,0.3,...,2.9,72.1,2.2,2.0,3.8,3.2,1.1,0.2,1.6,1
1,5342,3800,75,21.8,10.5,4.2,7.9,55.1,-0.3,-1.0,...,3.6,67.8,3.6,3.7,6.6,0.7,0.5,0.6,1.4,1
2,5716,3801,85,19.1,4.5,1.9,4.5,42.8,0.4,1.2,...,0.6,75.7,0.6,1.8,2.4,0.8,0.4,0.2,0.6,1
3,13790,3802,63,19.1,8.2,3.5,6.7,52.5,0.3,0.8,...,1.5,66.9,0.8,2.0,3.0,1.8,0.4,0.1,1.9,1
4,5470,3803,63,17.8,3.7,1.7,3.4,50.8,0.5,1.4,...,0.5,54.0,2.4,2.7,4.9,0.4,0.4,0.6,0.7,1


In [737]:
test_raw.head()

,Id_old,Id,GP,MIN,PTS,FGM,FGA,FG%,3P Made,3PA,...,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV
0,1,0,56,9.1,4.0,1.6,3.7,43.7,0.1,0.3,...,0.7,1.2,63.4,1.2,0.8,1.7,0.4,0.2,0.3,0.8
1,8194,1,43,19.3,10.1,3.7,8.1,46.0,0.6,1.7,...,1.8,2.5,75.3,0.5,0.9,1.5,3.5,0.6,0.0,1.8
2,3,2,82,33.9,11.3,4.9,10.6,45.6,0.5,1.9,...,1.8,2.7,71.2,1.3,3.3,4.5,2.5,1.3,0.3,2.0
3,8196,3,86,44.7,18.8,6.8,15.9,42.9,0.5,1.8,...,4.5,6.3,70.9,1.5,3.2,5.0,4.1,0.9,0.1,3.6
4,8197,4,58,12.3,4.7,1.6,4.0,40.0,0.5,1.7,...,1.1,1.3,76.9,0.2,0.6,0.9,1.5,0.5,-0.4,0.9


In [738]:
# info
train_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Id_old       8000 non-null   int64  
 1   Id           8000 non-null   int64  
 2   GP           8000 non-null   int64  
 3   MIN          8000 non-null   float64
 4   PTS          8000 non-null   float64
 5   FGM          8000 non-null   float64
 6   FGA          8000 non-null   float64
 7   FG%          8000 non-null   float64
 8   3P Made      8000 non-null   float64
 9   3PA          8000 non-null   float64
 10  3P%          8000 non-null   float64
 11  FTM          8000 non-null   float64
 12  FTA          8000 non-null   float64
 13  FT%          8000 non-null   float64
 14  OREB         8000 non-null   float64
 15  DREB         8000 non-null   float64
 16  REB          8000 non-null   float64
 17  AST          8000 non-null   float64
 18  STL          8000 non-null   float64
 19  BLK   

In [739]:
print(train_raw.where(train_raw < 0).count())

Id_old            0
Id                0
GP                2
MIN               0
PTS               0
FGM               0
FGA               0
FG%               0
3P Made        1629
3PA            1658
3P%             878
FTM               0
FTA               0
FT%               1
OREB              0
DREB              0
REB               0
AST               0
STL               0
BLK            1048
TOV               0
TARGET_5Yrs       0
dtype: int64


In [740]:
len(train_raw[(train_raw['3P%'] < 0) | (train_raw['3P Made'] < 0) | (train_raw['3PA'] < 0)])

2192

In [741]:
len(train_raw[(train_raw['3P%'] < 0) | (train_raw['3P Made'] < 0) | (train_raw['3PA'] < 0) | (train_raw['BLK'] < 0)])

3046

In [742]:
#variable descriptions
train_raw.describe()

,Id_old,Id,GP,MIN,PTS,FGM,FGA,FG%,3P Made,3PA,...,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,TARGET_5Yrs
count,8000.000000,8000.00000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,...,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000
mean,6856.971000,7798.50000,62.777875,18.576663,7.267087,2.807037,6.231213,44.608900,0.264525,0.816563,...,1.947787,71.365825,1.077838,2.168500,3.245300,1.624513,0.648688,0.245212,1.257762,0.833625
std,3977.447579,2309.54541,17.118774,8.935263,4.318732,1.693373,3.584559,6.155453,0.384093,1.060964,...,1.252352,10.430447,0.785670,1.392224,2.085154,1.355986,0.407626,0.821037,0.723270,0.372440
min,4.000000,3799.00000,-8.000000,2.900000,0.800000,0.300000,0.800000,21.300000,-1.100000,-3.100000,...,0.000000,-13.300000,0.000000,0.200000,0.300000,0.000000,0.000000,-17.900000,0.100000,0.000000
25%,3413.750000,5798.75000,51.000000,12.000000,4.100000,1.600000,3.600000,40.400000,0.000000,0.100000,...,1.000000,65.000000,0.500000,1.100000,1.700000,0.700000,0.300000,0.100000,0.700000,1.000000
50%,6787.500000,7798.50000,63.000000,16.800000,6.300000,2.400000,5.400000,44.400000,0.300000,0.800000,...,1.700000,71.400000,0.900000,1.900000,2.800000,1.300000,0.600000,0.200000,1.100000,1.000000
75%,10299.250000,9798.25000,74.000000,23.500000,9.500000,3.700000,8.100000,48.700000,0.500000,1.500000,...,2.600000,77.500000,1.500000,2.900000,4.300000,2.200000,0.900000,0.400000,1.600000,1.000000
max,13798.000000,11798.00000,123.000000,73.800000,34.200000,13.100000,28.900000,67.200000,1.700000,4.700000,...,11.100000,168.900000,5.500000,11.000000,15.900000,12.800000,3.600000,18.900000,5.300000,1.000000


In [743]:
test_raw.describe()

,Id_old,Id,GP,MIN,PTS,FGM,FGA,FG%,3P Made,3PA,...,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV
count,3799.000000,3799.000000,3799.000000,3799.000000,3799.000000,3799.000000,3799.000000,3799.000000,3799.000000,3799.000000,...,3799.000000,3799.000000,3799.000000,3799.000000,3799.000000,3799.000000,3799.000000,3799.000000,3799.000000,3799.000000
mean,7010.614109,1899.000000,62.853909,18.650224,7.328034,2.835404,6.302580,44.599079,0.255962,0.796920,...,1.399842,1.953567,71.612924,1.096025,2.179495,3.275783,1.636483,0.653593,0.257726,1.257910
std,3954.173641,1096.821164,17.151740,8.727259,4.294724,1.688427,3.579221,6.040168,0.380987,1.052862,...,0.926140,1.250376,10.457336,0.785678,1.371935,2.070646,1.335496,0.410573,0.639660,0.712449
min,1.000000,0.000000,6.000000,3.700000,0.700000,0.300000,0.800000,25.100000,-1.000000,-2.700000,...,0.000000,0.000000,23.700000,0.000000,0.200000,0.300000,0.000000,0.000000,-7.100000,0.100000
25%,3644.000000,949.500000,51.000000,12.200000,4.200000,1.600000,3.700000,40.500000,0.000000,0.100000,...,0.700000,1.000000,65.000000,0.500000,1.200000,1.800000,0.600000,0.400000,0.100000,0.700000
50%,7062.000000,1899.000000,63.000000,17.000000,6.400000,2.500000,5.500000,44.600000,0.300000,0.800000,...,1.200000,1.700000,71.500000,0.900000,1.900000,2.800000,1.300000,0.600000,0.200000,1.100000
75%,10402.500000,2848.500000,74.000000,23.300000,9.400000,3.700000,8.100000,48.500000,0.500000,1.500000,...,1.900000,2.600000,78.000000,1.500000,2.900000,4.300000,2.300000,0.900000,0.400000,1.600000
max,13792.000000,3798.000000,126.000000,68.000000,33.000000,13.400000,26.200000,74.600000,1.600000,4.300000,...,7.800000,9.800000,127.100000,6.900000,12.000000,18.500000,9.000000,2.700000,14.800000,5.200000


## Decisions

We will retain all potential features, when using non-linear models.

We will set negative values to 

and TARGET_5Yrs is our target.

## Cleaning

In [744]:
train = train_raw.copy()
test = test_raw.copy()

In [745]:
cols_drop = ['Id_old', 'Id'] #, 'MIN', 'FGM', 'FGA', 'TOV', '3PA', 'FTM', 'FTA', 'REB']
train.drop(cols_drop, axis=1, inplace=True)
test.drop(cols_drop, axis=1, inplace=True)

In [746]:
train.head()

,GP,MIN,PTS,FGM,FGA,FG%,3P Made,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,TARGET_5Yrs
0,80,24.3,7.8,3.0,6.4,45.7,0.1,0.3,22.6,2.0,2.9,72.1,2.2,2.0,3.8,3.2,1.1,0.2,1.6,1
1,75,21.8,10.5,4.2,7.9,55.1,-0.3,-1.0,34.9,2.4,3.6,67.8,3.6,3.7,6.6,0.7,0.5,0.6,1.4,1
2,85,19.1,4.5,1.9,4.5,42.8,0.4,1.2,34.3,0.4,0.6,75.7,0.6,1.8,2.4,0.8,0.4,0.2,0.6,1
3,63,19.1,8.2,3.5,6.7,52.5,0.3,0.8,23.7,0.9,1.5,66.9,0.8,2.0,3.0,1.8,0.4,0.1,1.9,1
4,63,17.8,3.7,1.7,3.4,50.8,0.5,1.4,13.7,0.2,0.5,54.0,2.4,2.7,4.9,0.4,0.4,0.6,0.7,1


In [747]:
test.head()

,GP,MIN,PTS,FGM,FGA,FG%,3P Made,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV
0,56,9.1,4.0,1.6,3.7,43.7,0.1,0.3,7.3,0.7,1.2,63.4,1.2,0.8,1.7,0.4,0.2,0.3,0.8
1,43,19.3,10.1,3.7,8.1,46.0,0.6,1.7,35.1,1.8,2.5,75.3,0.5,0.9,1.5,3.5,0.6,0.0,1.8
2,82,33.9,11.3,4.9,10.6,45.6,0.5,1.9,44.8,1.8,2.7,71.2,1.3,3.3,4.5,2.5,1.3,0.3,2.0
3,86,44.7,18.8,6.8,15.9,42.9,0.5,1.8,13.5,4.5,6.3,70.9,1.5,3.2,5.0,4.1,0.9,0.1,3.6
4,58,12.3,4.7,1.6,4.0,40.0,0.5,1.7,38.7,1.1,1.3,76.9,0.2,0.6,0.9,1.5,0.5,-0.4,0.9


In [748]:
# Negative values do not make sense in this context

def clean_negatives(strategy, df):
    
    if strategy=='abs':
        df = abs(df)
    if strategy=='null':
        df[df < 0] = None
    if strategy=='mean':
        df[df < 0] = None
        df.fillna(df.mean(), inplace=True)     
    if strategy=='median':
        df[df < 0] = None
        df.fillna(df.median(), inplace=True)   
    
    return(df)


In [749]:
negatives_strategy = 'abs'

train = clean_negatives(negatives_strategy, train)
test = clean_negatives(negatives_strategy, test)


In [750]:
train.head()

,GP,MIN,PTS,FGM,FGA,FG%,3P Made,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,TARGET_5Yrs
0,80.0,24.3,7.8,3.0,6.4,45.7,0.1,0.3,22.6,2.0,2.9,72.1,2.2,2.0,3.8,3.2,1.1,0.2,1.6,1.0
1,75.0,21.8,10.5,4.2,7.9,55.1,0.3,1.0,34.9,2.4,3.6,67.8,3.6,3.7,6.6,0.7,0.5,0.6,1.4,1.0
2,85.0,19.1,4.5,1.9,4.5,42.8,0.4,1.2,34.3,0.4,0.6,75.7,0.6,1.8,2.4,0.8,0.4,0.2,0.6,1.0
3,63.0,19.1,8.2,3.5,6.7,52.5,0.3,0.8,23.7,0.9,1.5,66.9,0.8,2.0,3.0,1.8,0.4,0.1,1.9,1.0
4,63.0,17.8,3.7,1.7,3.4,50.8,0.5,1.4,13.7,0.2,0.5,54.0,2.4,2.7,4.9,0.4,0.4,0.6,0.7,1.0


In [751]:
test.head()

,GP,MIN,PTS,FGM,FGA,FG%,3P Made,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV
0,56.0,9.1,4.0,1.6,3.7,43.7,0.1,0.3,7.3,0.7,1.2,63.4,1.2,0.8,1.7,0.4,0.2,0.3,0.8
1,43.0,19.3,10.1,3.7,8.1,46.0,0.6,1.7,35.1,1.8,2.5,75.3,0.5,0.9,1.5,3.5,0.6,0.0,1.8
2,82.0,33.9,11.3,4.9,10.6,45.6,0.5,1.9,44.8,1.8,2.7,71.2,1.3,3.3,4.5,2.5,1.3,0.3,2.0
3,86.0,44.7,18.8,6.8,15.9,42.9,0.5,1.8,13.5,4.5,6.3,70.9,1.5,3.2,5.0,4.1,0.9,0.1,3.6
4,58.0,12.3,4.7,1.6,4.0,40.0,0.5,1.7,38.7,1.1,1.3,76.9,0.2,0.6,0.9,1.5,0.5,0.4,0.9


In [752]:
train_target = train.pop('TARGET_5Yrs')

In [753]:
#examine shapes

print('train:' + str(train.shape))
print('test:' + str(test.shape))

train:(8000, 19)
test:(3799, 19)


In [754]:
# target class balance check
from collections import Counter

counter = Counter(train_target)
print(counter)

Counter({1.0: 6669, 0.0: 1331})


# Modelling

In [755]:
#transformations

# scaling - not for tree-based model


In [756]:
# set training data and validation data

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train, train_target, test_size=0.2, random_state=8)


In [757]:
#import models

!pip install xgboost
import xgboost as xgb


In [758]:
#Class balancing

class_weight = counter[1.0] / counter[0.0]
print(class_weight)


5.010518407212622


In [759]:
# Define model
initial_model = xgb.XGBClassifier(#scale_pos_weight = class_weight,
                                 use_label_encoder=False,
                                 seed=42)


In [760]:
# re-index the y_train, since xgboost has deprecated the use_label_encoder function

y_train.index = range(0, len(y_train), 1)


In [761]:
# Fit model

initial_model.fit(X_train, y_train)


[12:03:57] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=6, num_parallel_tree=1, random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42,
              subsample=1, tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbosity=None)

In [762]:
#dump(initial_model,  '../models/aj_' + experiment_label + '_initial.joblib')

In [763]:
y_train_preds = initial_model.predict(X_train)
y_val_preds = initial_model.predict(X_val)

In [764]:
def eval_report(X, true, pred):
    
    import numpy as np
    import pandas as pd
    from sklearn import metrics

    unique_label = np.unique([true, pred])
    cmtx = pd.DataFrame(
        metrics.confusion_matrix(true, pred, labels=unique_label), 
        index=['true:{:}'.format(x) for x in unique_label], 
        columns=['pred:{:}'.format(x) for x in unique_label]
    )
    print("Confusion matrix:")
    print(cmtx)
    print("")
    print(metrics.classification_report(true, pred))
    print("")
    print('ROC AUC: {:.4f}'.format(metrics.roc_auc_score(true, pred), ))


In [765]:
# model performance on training data

eval_report(X_train, y_train, y_train_preds)

Confusion matrix:
          pred:0.0  pred:1.0
true:0.0       926       148
true:1.0         0      5326

              precision    recall  f1-score   support

         0.0       1.00      0.86      0.93      1074
         1.0       0.97      1.00      0.99      5326

    accuracy                           0.98      6400
   macro avg       0.99      0.93      0.96      6400
weighted avg       0.98      0.98      0.98      6400


ROC AUC: 0.9311


In [766]:
# model performance on validation data

eval_report(X_val, y_val, y_val_preds)

Confusion matrix:
          pred:0.0  pred:1.0
true:0.0        23       234
true:1.0        56      1287

              precision    recall  f1-score   support

         0.0       0.29      0.09      0.14       257
         1.0       0.85      0.96      0.90      1343

    accuracy                           0.82      1600
   macro avg       0.57      0.52      0.52      1600
weighted avg       0.76      0.82      0.78      1600


ROC AUC: 0.5239


## Model tuning

In [767]:
# class imbalance pipeline

!pip install imblearn

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

In [ ]:
# Define model

xgboost_defaults = xgb.XGBClassifier(
    
    #booster params
    learning_rate = 0.3, # [0,1] increase to shrink feature weights at each step, make more conservative
    gamma = 0,      # [0, inf] min loss reduction required to make another split, bigger = higher hurdle, fewer splits
    max_depth = 6,  # [0, inf] reduce if overfitting
    min_child_weight = 1,  # [0, inf] related to number of obs in a node
    max_delta_step = 0,  # [0, inf] usually not needed, but sometimes increasing can improve very imbalanced classes
    subsample = 1,  # [0, 1] proportion of samples used in each boosting iteration, usu over 0.5 unless using 
    sampling_method = 'uniform', # other option is gradient_based, whcih requires tree_method = 'gpu_hist'
    colsample_bytree = 1,
    colsample_bylevel = 1,
    colsample_bynode = 1,
    reg_lambda = 1, # L2 reg
    alpha = 0,      # L1 reg
    tree_method = 'auto',
    scale_pos_weight = 1,
    num_parallel_tree = 1,
    
    #learning task params
    objective = 'binary:logistic', # consider rank:pairwise, multi:softprob (needs num_class(2))
    eval_metric = 'auc',
    use_label_encoder=False,
    seed=42
)


# list attributes of model
#model['model'].__dict__

In [ ]:

from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score

# Define cv
cv = RepeatedStratifiedKFold(
    n_splits=10, n_repeats=3, random_state=8
)


## Manual hyperparamter exploration

In [ ]:
def examine_parameter(overrides, search_param, search_values):
    
    trial_model = Pipeline([
        #('over', SMOTE()),    
        #('over', RandomOverSampler(sampling_strategy=0.25)),
        #('under', RandomUnderSampler()),
        ('model', xgboost_defaults)
    ])
    for o in overrides:
        setattr(trial_model['model'], o[0], o[1])
    print(trial_model['model'].__dict__)
    for v in search_values:
        setattr(trial_model['model'], search_param, v)
        cv_result = cross_val_score(trial_model, X_train, y_train, cv=cv, scoring="roc_auc", verbose=0)
        print('{}={} CV ROC AUC, mean and sd: {:.4f}, {:.4f}'.format(search_param, v, cv_result.mean(),cv_result.std()))


In [ ]:
overrides = [('scale_pos_weight', 5),
            ('use_label_encoder', False),
            ('objective', 'binary:logistic'),
            ('eval_metric', 'auc'),
            ('seed', 42)
            ]


In [ ]:
examine_parameter(overrides, 'max_depth', [2,3,4,5,6])


In [ ]:
overrides.append(('max_depth', 4))
examine_parameter(overrides, 'gamma', np.linspace(0, 0.5, 5))


In [ ]:
overrides.append(('gamma', 0))
examine_parameter(overrides, 'learning_rate', np.linspace(0.01, 0.15, 15))


In [ ]:
overrides.append(('learning_rate', 0.1))
examine_parameter(overrides, 'num_parallel_tree', [1,2,3,4,5])


In [ ]:
overrides.append(('num_parallel_tree', 1))
examine_parameter(overrides, 'min_child_weight', [1,2,3,4])


In [ ]:
overrides.append(('min_child_weight', 2))
examine_parameter(overrides, 'subsample', [1, 0.8, 0.6, 0.4])


In [ ]:
overrides.append(('subsample', 1))
examine_parameter(overrides, 'n_estimators', [80, 90, 100, 110, 120])


In [ ]:
print(range(3, 10, 1))

## Hyperparameter optimisation search

In [768]:
from hyperopt import Trials, STATUS_OK, tpe, hp, fmin
from hyperopt.pyll.base import scope


In [769]:
def objective(space):
    from sklearn.model_selection import cross_val_score
      
    xgboost = xgb.XGBClassifier(
        max_depth = int(space['max_depth']),
        learning_rate = space['learning_rate'],
        min_child_weight = space['min_child_weight'],
        gamma = space['gamma'],
        subsample = space['subsample'],
        colsample_bytree = space['colsample_bytree'],
        n_estimators = space['n_estimators'],
        scale_pos_weight = 5,
        use_label_encoder = False,
        objective = 'binary:logistic',
        eval_metric = 'auc',
        seed = 42
    )
    
    # Define model pipeline
    model = Pipeline([
        #('over', SMOTE(sampling_strategy=0.5)),
        #('under', RandomUnderSampler(sampling_strategy=0.75)),
        ('model', xgboost)
    ])
    
    acc = cross_val_score(model, X_train, y_train, cv=5, scoring="roc_auc").mean()

    return{'loss': 1-acc, 'status': STATUS_OK }

In [770]:
space = {
    'max_depth' : scope.int(hp.quniform('max_depth', 3, 10, 1)),
    'learning_rate' : hp.quniform('learning_rate', 0.04, 0.15, 0.01),
    'min_child_weight' : hp.quniform('min_child_weight', 1, 8, 1),
    'gamma' : hp.quniform('gamma', 0, 0.05, 0.01),
    'subsample' : hp.quniform('subsample', 0.1, 1, 0.1),
    'colsample_bytree' : hp.quniform('colsample_bytree', 0.1, 1.0, 0.05),
    'n_estimators' : scope.int(hp.quniform('n_estimators', 80, 120, 10))
}

In [771]:
best = fmin(
    fn=objective,   
    space=space,       
    algo=tpe.suggest,       
    max_evals=30
)


100%|██████████| 30/30 [00:21<00:00,  1.41trial/s, best loss: 0.30108821466431945]


In [772]:
print("Best: ", best)

Best:  {'colsample_bytree': 0.6000000000000001, 'gamma': 0.01, 'learning_rate': 0.05, 'max_depth': 3.0, 'min_child_weight': 8.0, 'n_estimators': 120.0, 'subsample': 0.8}


## Fit best model

In [773]:
### and here is where we take the best hyperparamters from the hyperparameter search and fit the model again.

xgboost_2 = xgb.XGBClassifier(
    max_depth = int(best['max_depth']),
    learning_rate = best['learning_rate'],
    min_child_weight = int(best['min_child_weight']),
    gamma = best['gamma'],    
    subsample = best['subsample'],
    colsample_bytree = best['colsample_bytree'],
    n_estimators = int(best['n_estimators']),
    scale_pos_weight = 5,
    use_label_encoder = False,
    objective = 'binary:logistic',
    eval_metric = 'auc',
    seed = 42
)

# Define model pipeline
best_model = Pipeline([
    #('over', SMOTE(sampling_strategy=0.5)),
    #('under', RandomUnderSampler(sampling_strategy=0.75)),
    ('model', xgboost_2)
])

best_model.fit(X_train, y_train)


Pipeline(steps=[('model',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=0.6000000000000001,
                               eval_metric='auc', gamma=0.01, gpu_id=-1,
                               importance_type='gain',
                               interaction_constraints='', learning_rate=0.05,
                               max_delta_step=0, max_depth=3,
                               min_child_weight=8, missing=nan,
                               monotone_constraints='()', n_estimators=120,
                               n_jobs=6, num_parallel_tree=1, random_state=42,
                               reg_alpha=0, reg_lambda=1, scale_pos_weight=5,
                               seed=42, subsample=0.8, tree_method='exact',
                               use_label_encoder=False, validate_parameters=1,
                               verbosity=None))]

In [774]:
#save model to file

#dump(best_model,  '../models/aj_' + experiment_label + '_best.joblib')

In [775]:
#predictions for train & validation sets

y_train_preds = best_model.predict(X_train)
y_val_preds = best_model.predict(X_val)

## Evaluation

In [776]:
# model performance on training data

eval_report(X_train, y_train, y_train_preds)

Confusion matrix:
          pred:0.0  pred:1.0
true:0.0         0      1074
true:1.0         0      5326

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00      1074
         1.0       0.83      1.00      0.91      5326

    accuracy                           0.83      6400
   macro avg       0.42      0.50      0.45      6400
weighted avg       0.69      0.83      0.76      6400


ROC AUC: 0.5000


/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# model performance on validation data

eval_report(X_val, y_val, y_val_preds)

In [ ]:
# ROC curves

import matplotlib.pyplot as plt  
from sklearn import metrics
metrics.plot_roc_curve(best_model, X_train, y_train)
plt.show() 
metrics.plot_roc_curve(best_model, X_val, y_val)
plt.show() 

# Apply to test data for submission

In [ ]:
y_test_preds = best_model.predict(test)


In [ ]:
y_test_probs = best_model.predict_proba(test)

In [ ]:
submission = pd.DataFrame({'Id': range(0,3799), 'TARGET_5Yrs': [p[1] for p in y_test_probs]})

In [ ]:
submission.head()

In [ ]:
submission.to_csv('../reports/aj_' + experiment_label + 'submission.csv',
                 index=False,
                 )